# Interactive Editor for 'metarisk_releases_dim'

This notebook provides a best-in-class, production-ready template for an interactive table editor in Databricks, specifically configured for the `metarisk_releases_dim` table. It is designed for small tables (<200 rows) and is compatible with Databricks Runtime 17.1 ML.

### Usage Guide

*   **Author:** Eric Regna*   **Generated:** 2025-09-30 20:20:39 UTC
*   **Purpose:** This notebook provides a user-friendly interface to directly edit records in the `gc_prod_sandbox.su_eric_regna.metarisk_releases_dim` table.

**How to Use This Editor:**

1.  **Run Initial Cells:** Execute the cells in sections 1 through 5 to install libraries, configure the connection, and display the interactive grid below.
2.  **Choose Your Data Source (Section 3):**
    *   By default, the notebook loads **sample data** for safe testing. You can edit this data and test the "Save" functionality without affecting the live table (the save will be simulated).
    *   To edit the **live Databricks table**, go to the cell in Section 3, comment out the `df = generate_realistic_sample_data()` line, and uncomment the `df = load_data_from_databricks(...)` line. Then, re-run that cell.
3.  **Edit Data in the Grid:**
    *   **Text/Date/Dropdowns:** Double-click a cell in an editable column (`Status`, `Product`, `start_date`, `Callouts`) to activate the editor.
    *   **Multi-Select (`release_labels`):** Double-click a cell in the `release_labels` column. A popup with checkboxes will appear. Select or deselect items and click outside the popup to confirm.
    *   **Multi-Line Text (`Callouts`):** Double-click a cell. A larger text box will appear. You can use `Enter` to create new lines. Click outside the box to finish.
4.  **Save Your Changes:**
    *   After making all your edits, you **MUST** click the green **"Save Changes"** button.
    *   This action takes all your changes from the grid and applies them to the live Databricks table using a `MERGE` command. You will see a confirmation message and the SQL command that was executed in the output area.
5.  **Refresh Data:**
    *   Click the blue **"Refresh Data"** button to discard any unsaved changes in the grid and reload the latest data from the Databricks table.

## 1. Installation and Setup

In [ ]:
%pip install ipyaggrid==0.3.0 pandas==1.5.3

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time

from ipyaggrid import Grid
from ipywidgets import Button, HBox, VBox, HTML, Output
from IPython.display import display, Javascript

# The SparkSession is automatically available in a Databricks notebook environment
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

## 2. Configuration for `metarisk_releases_dim`

In [ ]:
# --- REAL CONFIGURATION --- #
CATALOG = "gc_prod_sandbox"
SCHEMA = "su_eric_regna"
TABLE = "metarisk_releases_dim"
TABLE_FQN = f"{CATALOG}.{SCHEMA}.{TABLE}"

# Define all columns you want to display in the grid
# Assuming 'id' is the primary key and 'release_name' exists for context
COLUMNS_TO_DISPLAY = ["id", "release_name", "Status", "Product", "start_date", "release_labels", "Callouts"]
PRIMARY_KEY = "id"

## 3. Data Loading and Sample Data Generation

This section contains two functions: one to generate sample data for quick testing, and one to load the actual data from your Databricks table. By default, the sample data is used.

In [ ]:
def generate_realistic_sample_data():
    """Generates a sample DataFrame that mimics your table for testing purposes."""
    status_opts = ["On-Track", "At-Risk", "Off-Track", "Complete", "Not Started", "Blocked"]
    product_opts = ["MR Desktop", "Data", "MR Online", "MR Live", "Support", "MR Rate", "AI"]
    labels_opts = ["Engine", "Global Tools", "Hotfix", "ModelBuilder", "Rating Tools", "Reinsurance", "Templates", "UI/UX ", "Waterfalls", "AI"]
    
    data = {
        'id': range(101, 111),
        'release_name': [f'2025.Q{i} Release' for i in range(1, 11)],
        'Status': np.random.choice(status_opts, 10),
        'Product': np.random.choice(product_opts, 10),
        'start_date': [(datetime.now() - timedelta(days=i*30)).strftime('%Y-%m-%d') for i in range(10)],
        'release_labels': [';'.join(np.random.choice(labels_opts, size=np.random.randint(1, 4), replace=False)) for _ in range(10)],
        'Callouts': [f'Key callout for release {i}.\n- Point 1\n- Point 2' for i in range(1, 11)]
    }
    return pd.DataFrame(data)

def load_data_from_databricks(table_fqn, columns):
    """Loads data from the specified Databricks table into a pandas DataFrame."""
    print(f"Loading data from {table_fqn}...")
    try:
        # Select specific columns and limit to 200 rows as per the requirement
        df_spark = spark.table(table_fqn).select(*columns).limit(200)
        df_pandas = df_spark.toPandas()
        
        # Ensure date columns are in string format for the grid editor
        if 'start_date' in df_pandas.columns:
            df_pandas['start_date'] = pd.to_datetime(df_pandas['start_date']).dt.strftime('%Y-%m-%d')
        
        print("Data loaded successfully.")
        return df_pandas
    except Exception as e:
        print(f"ERROR: Could not load data from {table_fqn}. Please check table name and permissions. Error: {e}")
        return pd.DataFrame(columns=columns)

# --- CHOOSE DATA SOURCE --- #
# Use sample data for testing
df = generate_realistic_sample_data()

# Uncomment the line below to use your actual Databricks table
# df = load_data_from_databricks(TABLE_FQN, COLUMNS_TO_DISPLAY)

## 4. Configure Grid with Custom Multi-Select Editor

Here we define the behavior of each column in the grid. A custom JavaScript class is created on-the-fly for the `release_labels` multi-select editor. The `agLargeTextCellEditor` is used for `Callouts` to allow multi-line input.

In [ ]:
custom_multi_select_editor_js = """
class MultiSelectCellEditor {
    init(params) {
        this.values = params.values;
        this.eGui = document.createElement('div');
        this.eGui.style.backgroundColor = 'white';
        this.eGui.style.border = '1px solid #aaa';
        this.eGui.style.padding = '5px';
        this.eGui.style.boxShadow = '2px 2px 5px rgba(0,0,0,0.1)';
        const currentValues = params.value ? params.value.split(';') : [];
        this.values.forEach(value => {
            const label = document.createElement('label');
            label.style.display = 'block';
            const checkbox = document.createElement('input');
            checkbox.type = 'checkbox';
            checkbox.value = value;
            checkbox.checked = currentValues.includes(value);
            label.appendChild(checkbox);
            label.appendChild(document.createTextNode(' ' + value));
            this.eGui.appendChild(label);
        });
    }
    getGui() { return this.eGui; }
    afterGuiAttached() { this.eGui.focus(); }
    getValue() {
        const selected = this.eGui.querySelectorAll('input[type="checkbox"]:checked');
        return Array.from(selected).map(cb => cb.value).join(';');
    }
    isPopup() { return true; }
    destroy() {}
}
"""

def configure_grid_columns():
    """Configure grid columns with appropriate editors and settings for the metarisk table."""
    return [
        {'headerName': 'ID', 'field': 'id', 'editable': False, 'width': 80},
        {'headerName': 'Release Name', 'field': 'release_name', 'editable': False, 'width': 180},
        {
            'headerName': 'Status',
            'field': 'Status',
            'editable': True,
            'cellEditor': 'agSelectCellEditor',
            'cellEditorParams': {'values': ["On-Track", "At-Risk", "Off-Track", "Complete", "Not Started", "Blocked"]},
            'width': 130
        },
        {
            'headerName': 'Product',
            'field': 'Product',
            'editable': True,
            'cellEditor': 'agSelectCellEditor',
            'cellEditorParams': {'values': ["MR Desktop", "Data", "MR Online", "MR Live", "Support", "MR Rate", "AI"]},
            'width': 130
        },
        {
            'headerName': 'Start Date',
            'field': 'start_date',
            'editable': True,
            'cellEditor': 'agDateCellEditor',
            'width': 130
        },
        {
            'headerName': 'Release Labels',
            'field': 'release_labels',
            'editable': True,
            'cellEditor': 'MultiSelectCellEditor', // Our custom multi-select editor
            'cellEditorParams': {
                'values': ["Engine", "Global Tools", "Hotfix", "ModelBuilder", "Rating Tools", "Reinsurance", "Templates", "UI/UX ", "Waterfalls", "AI"]
            },
            'width': 250
        },
        {
            'headerName': 'Callouts',
            'field': 'Callouts',
            'editable': True,
            'cellEditor': 'agLargeTextCellEditor', // Supports multi-line input
            'cellEditorParams': {'rows': 5, 'cols': 40},
            'cellStyle': {'white-space': 'pre-wrap'}, // Ensures line breaks are displayed in the grid
            'autoHeight': True, // Adjusts row height to fit content
            'width': 350
        }
    ]


## 5. Create and Display the Grid

In [ ]:
column_defs = configure_grid_columns()

grid_options = {
    'columnDefs': column_defs,
    'defaultColDef': {
        'filter': True,
        'sortable': True,
        'resizable': True
    },
    'enableCellChangeFlash': True,
    'undoRedoCellEditing': True,
    'stopEditingWhenCellsLoseFocus': True, // Crucial for sync reliability
    'components': {
        'MultiSelectCellEditor': Javascript(custom_multi_select_editor_js)
    }
}

grid = Grid(
    grid_data=df,
    grid_options=grid_options,
    height=500,
    width='100%',
    theme='ag-theme-alpine',
    quick_filter=True,
    show_toggle_edit=True,
    sync_on_edit=False, # Disable auto-sync; we use a manual save button
    export_csv=True,
    export_excel=True
)

## 6. Handle Data Changes and Save to Databricks

This section implements the robust "manual save" pattern. When the user clicks "Save Changes", we stop all editing, pull the complete dataset from the grid, and then perform the `MERGE` operation against your live table.

In [ ]:
output_area = Output()
save_button = Button(description="Save Changes", button_style="success", icon="save")
refresh_button = Button(description="Refresh Data", button_style="info", icon="refresh")
status_html = HTML("<p>Grid is ready. Edit cells and click 'Save Changes'.</p>")

def on_save_clicked(b):
    with output_area:
        output_area.clear_output()
        status_html.value = f"<p style='color:blue'>Saving... Please wait.</p>"
        try:
            # 1. Stop any active editing in the grid (CRITICAL STEP)
            grid.js_code("gridOptions.api.stopEditing()");
            
            # 2. Add a small delay for the front-end to process the change
            time.sleep(0.5)
            
            # 3. Get the updated data directly from the grid's front-end state
            updated_df = grid.grid_data_out
            
            print("Data to be saved:")
            display(updated_df.head())
            
            # --- DATABRICKS INTEGRATION --- #
            print(f"\nConverting to Spark DataFrame and merging into {TABLE_FQN}...")
            updated_spark_df = spark.createDataFrame(updated_df)
            updated_spark_df.createOrReplaceTempView("temp_updates_for_merge")
            
            # Construct the MERGE statement dynamically based on editable columns
            editable_cols = [c['field'] for c in configure_grid_columns() if c.get('editable')]
            set_clauses = ",\n".join([f"  target.{col} = source.{col}" for col in editable_cols])
            
            merge_sql = f"""
            MERGE INTO {TABLE_FQN} AS target
            USING temp_updates_for_merge AS source
            ON target.{PRIMARY_KEY} = source.{PRIMARY_KEY}
            WHEN MATCHED THEN UPDATE SET
            {set_clauses}
            """
            # WHEN NOT MATCHED THEN INSERT *  <- Optional: Add if you want to insert new rows from grid
            
            print("Executing MERGE statement:")
            print(merge_sql)
            spark.sql(merge_sql)
            # ------------------------------ #
            
            status_html.value = f"<p style='color:green'>Data saved to {TABLE_FQN} successfully at {datetime.now().strftime('%H:%M:%S')}</p>"
            print("\nSave operation completed.")

        except Exception as e:
            error_message = f"Error saving data: {str(e)}"
            print(error_message)
            status_html.value = f"<p style='color:red'>{error_message}</p>"

def on_refresh_clicked(b):
    with output_area:
        output_area.clear_output()
        status_html.value = f"<p style='color:blue'>Refreshing data from {TABLE_FQN}...</p>"
        try:
            refreshed_df = load_data_from_databricks(TABLE_FQN, COLUMNS_TO_DISPLAY)
            grid.grid_data = refreshed_df
            grid.js_code("gridOptions.api.setRowData(gridOptions.rowData)") # Safely reloads data in the grid
            status_html.value = f"<p style='color:green'>Grid data refreshed at {datetime.now().strftime('%H:%M:%S')}</p>"
        except Exception as e:
            error_message = f"Error refreshing data: {str(e)}"
            print(error_message)
            status_html.value = f"<p style='color:red'>{error_message}</p>"

save_button.on_click(on_save_clicked)
refresh_button.on_click(on_refresh_clicked)

controls = VBox([
    HBox([save_button, refresh_button]),
    status_html,
    output_area
])

display(VBox([grid, controls]))